In [2]:
!pip install fredapi

In [22]:
from dataclasses import dataclass, field
from typing import Optional
from datetime import datetime, timedelta
import yfinance as yf
from fredapi import Fred
import pandas as pd

fred = Fred(api_key='5079f41d061a4037d81f3da69e018803')

## Engine : Yield curve interpolation for cash flow discounting
## 

In [ ]:
end_date, start_date = _most_recent_business_day()
end_date.isoformat()

'2025-11-28'

In [56]:
frequency_mapping_dict = {
    'Annually': 1,
    'Semi-Annually': 2,
    'Quarterly': 4
}

@dataclass 
class Stock:
    Ticker: str

    def price(self):
        return yf.Ticker(self.Ticker).fast_info['lastPrice']

@dataclass
class Interest_Rates:
    Maturity: str
    Frequency: str

    Num_Frequency: int = field(init=False)
    T: int = field(init=False)

    def __post_init__(self):
        try:
            self.Num_Frequency = frequency_mapping_dict[self.Frequency]
        except KeyError:
            raise ValueError(f'Incorrect Frequency: {self.Frequency} should be one of {list(frequency_mapping_dict.keys())}')

        try:
            self.T = (datetime.strptime(self.Maturity, '%Y-%m-%d')-datetime.today()).days/365
        except ValueError:
            raise ValueError(f'Incorrect date : {self.Maturity}. Please use YYYY-MM-DD format')

    def _most_recent_business_day():
        ref_date = datetime.today().date()
        while ref_date.weekday() >= 5: 
            ref_date -= timedelta(days=1)
        previous_bd = ref_date-timedelta(days=1)
        data = pd.Series()
        while len(data)==0:
            data = fred.get_series('DGS1', observation_start= previous_bd, observation_end = last_bd)
            previous_bd -= timedelta(days=1)
        
        return ref_date.isoformat(), previous_bd.isoformat()



    def _get_yield_data(self, series_id):
        end_date, start_date = _most_recent_business_day()
        data = fred.get_series(series_id, observation_start=start_date, observation_end=start_date)
        return data

    def yield_curve_nelson_siegel(self):
        treasuries_ids = ['DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3', 'DGS5', \
              'DGS7', 'DGS10', 'DGS20', 'DGS30']
        yields_dict = {t_id : self._get_yield_data(t_id) for t_id in treasuries_ids}
        return yields_dict

    



    

@dataclass
class Convertible:
    Face_Value: float
    Coupon: float
    Coupon_Frequency : str
    Conversion_Ratio : float
    Conversion_Strike : float 

    First_Coupon_Date : str
    Maturity: str
    Redemption : float
    Underlying_stock_ticker : str
    Day_Count_Convention : Optional[str] = 'ACT/ACT'
    Bond_Currency : Optional[str] = 'EUR'
    Stock_Currency : Optional[str] = 'EUR'
    AI_on_Conversion: Optional[bool] = True

    def conversion_price(self):
        return self.Face_Value/self.Conversion_Ratio
    
    def parity(self):
        #Equivalent value in shares if converted right now
        return None

@dataclass
class Option:
    Underlying: str
    K : float
    T : str  
    r = yf.Ticker('e')


In [57]:
Interest_Rates('2026-11-30','Annually').yield_curve_nelson_siegel()

{'DGS1MO': 2025-11-25    4.04
 dtype: float64,
 'DGS3MO': 2025-11-25    3.9
 dtype: float64,
 'DGS6MO': 2025-11-25    3.75
 dtype: float64,
 'DGS1': 2025-11-25    3.6
 dtype: float64,
 'DGS2': 2025-11-25    3.43
 dtype: float64,
 'DGS3': 2025-11-25    3.46
 dtype: float64,
 'DGS5': 2025-11-25    3.55
 dtype: float64,
 'DGS7': 2025-11-25    3.77
 dtype: float64,
 'DGS10': 2025-11-25    4.01
 dtype: float64,
 'DGS20': 2025-11-25    4.62
 dtype: float64,
 'DGS30': 2025-11-25    4.67
 dtype: float64}